In [1]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit


### Install adk

In [2]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00


In [3]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
# MODEL_NAME = "gemini-2.0-flash"
MODEL_NAME = "gemini-2.5-pro-preview-03-25"

BUCKET_URI = f"gs://agent-0417"

#  model = "gemini-2.5-pro-preview-03-25"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initial set up

In [4]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [5]:
# @title Initialize Vertex AI with Staging Bucket.
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Agent on adk

In [32]:
#@title Build a Agent with adk.
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_assistant",
    model=MODEL_NAME,
    instruction="You are a helpful assistant. Answer user questions using Google Search when needed.",
    description="An assistant that can search the web.",
    tools=[google_search] # Use Google search tool
)


In [33]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
   user_id="shins",  # Required
   message="What is the exchange rate from US dollars to Korean currency today ?",
):
   print(event)
   print(event['content']['parts'][0]['text'])

{'content': {'parts': [{'text': 'As of Friday, April 18, 2025, the exchange rate between US dollars (USD) and South Korean Won (KRW) is approximately:\n\n*   **1 USD = 1,417.52 KRW** according to Xe.com.\n*   **1 USD = 1,421.82 KRW** also reported by Xe.com in another snippet (rates can fluctuate slightly).\n*   One source mentions the rate increased to **1,417.15 KRW** on Thursday, April 17th.\n\nPlease note that exchange rates fluctuate constantly. The exact rate you get will depend on when and where you exchange the currency (e.g., bank, currency exchange service, money transfer provider). Some services like Remitly might offer promotional rates for first-time users (e.g., 1 USD = 1424.70 KRW for the first $500 sent).'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'tradingeconomics.com', 'title': 'tradingeconomics.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJyVlFGKTJ-FwlXddEFRsk3F5p8YYYb6Pjc-ZmoE2oJ1T7vkr58

## Use a Tool

In [27]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "KRW" (KRW).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [28]:
get_exchange_rate(currency_from="USD", currency_to="KRW")


{'amount': 1.0, 'base': 'USD', 'date': '2025-04-17', 'rates': {'KRW': 1416.48}}

In [29]:
from google.adk.agents import Agent

agent = Agent(
    model=MODEL_NAME,
    name='currency_exchange_agent',
    tools=[get_exchange_rate],
)

In [30]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won on 2025-04-17?",
):
   print(event)


{'content': {'parts': [{'text': 'I cannot provide future exchange rates. The date you requested, 2025-04-17, is in the future.\n\nWould you like me to get the latest available exchange rate for USD to KRW instead? Or perhaps the rate for a specific date in the past?'}], 'role': 'model'}, 'invocation_id': 'e-8e932d45-8a40-4bdb-8ba0-6d5821ca94b2', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '8ikioWpf', 'timestamp': 1744963767.298644}


## Manage sessions

In [12]:
def session_service_builder():
  from google.adk.sessions import InMemorySessionService

  return InMemorySessionService()

In [13]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(
   agent=agent,                                      # Required.
   session_service_builder=session_service_builder,  # Optional.
)

In [14]:
# session = app.create_session(user_id="shins")


In [15]:
# app.list_sessions(user_id="shins")


ListSessionsResponse(sessions=[Session(id='a816404b-872e-43a6-ac76-a45c81182e35', app_name='default-app-name', user_id='shins', state={}, events=[], last_update_time=1744963433.129747)])

In [ ]:
# session_id="a816404b-872e-43a6-ac76-a45c81182e35"

# session = app.get_session(user_id="shins", session_id=session_id)
# print(session)

In [ ]:
for event in app.stream_query(
    user_id="shins",
    # session_id=session_id, # Optional. you can pass in the session_id when querying the agent
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-15?",
):
    print(event)

In [34]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    app,
    display_name="currency agent with adk",
    gcs_dir_name = "ai-agent-adk",
    description="This is a simple agent with adk on agent engine.",
    requirements=[
        "google-adk",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.88.0', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-adk', 'cloudpickle==3.0']
INFO:vertexai.agent_engines:Using bucket agent-0417
INFO:vertexai.agent_engines:Wrote to gs://agent-0417/ai-agent-adk/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/8720807262030921728/operations/1574980690879971328
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=ai-hangsik
INFO:vertexai.agent_engines:AgentEngine created. Resource

In [38]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-15?",
):
    print(event)

{'content': {'parts': [{'text': 'Based on the search results for the USD to KRW exchange rate on April 15, 2025, here are the findings:\n\n*   One source indicates that on **Tuesday, April 15, 2025, 1 US Dollar (USD) was equal to 1428 South Korean Won (KRW)**. It also mentions a range with 1421 and 1429 KRW for that day.\n*   Another source (Investing.com) lists the closing price for April 15, 2025, as **1 USD = 1,413.71 KRW**. It also provides the opening rate (1,427.94 KRW), the high (1,432.74 KRW), and the low (1,412.39 KRW) for that day.\n*   A third source (XE.com) shows a rate of **1 USD = 1,428.11 KRW** around 22:02 UTC on April 15, 2025, and another rate of **1 USD = 1,427.95 KRW** at 23:25 UTC on the same day.\n*   A fourth source (Foreign Exchange UK) lists **1 USD = 1427.9909 KRW** for April 15, 2025.\n*   OFX provides a historical average rate of **1 USD = 1452.0046 KRW** for April 15, 2025.\n*   YCharts indicates a rate of **0.0007 USD to 1 KRW** for April 15, 2025, which 

## Agent Management

In [39]:
# @title Helper function to manage reasoning engine.

# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview

#----------------------------------------
def get_agent_engine(display_name:str):
  try:
    for agent in agent_engines.list():
      if agent.display_name == display_name:
        return agent_engines.get(agent.name)

      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def show_agents():
  """
  List reasoning engines.
  """

  try:
    if not agent_engines.list():
      print("No reasoning engines")

    for idx, agent in enumerate(agent_engines.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{agent.display_name}] \n\tName [{agent.name}] \n\tCreation Time [{agent.create_time}] \n\tResource Name [{agent.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def delete_agent(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = agent_engines.get(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


In [40]:
show_agents()

Agent 0: 
	Display Name [currency agent with adk] 
	Name [8720807262030921728] 
	Creation Time [2025-04-18 08:10:36.636926+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/8720807262030921728]

Agent 1: 
	Display Name [currency agent with adk] 
	Name [3726315275277041664] 
	Creation Time [2025-04-18 08:05:28.788186+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/3726315275277041664]

Agent 2: 
	Display Name [currency agent] 
	Name [1965407820975177728] 
	Creation Time [2025-04-16 04:31:34.165206+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/1965407820975177728]

Agent 3: 
	Display Name [currency agent with adk] 
	Name [4968182872524455936] 
	Creation Time [2025-04-16 03:28:46.545118+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/4968182872524455936]



In [41]:
get_agent_engine(display_name="currency agent with adk")


resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/8720807262030921728

In [ ]:
delete_agent(name="5350988840850948096")

In [43]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-14?",
):
    print(event)

{'content': {'parts': [{'text': 'Based on the search results, here are the reported USD to KRW (US Dollar to South Korean Won) exchange rates for April 14, 2025:\n\n*   **Investing.com:**\n    *   Closing Price: 1 USD = 1,426.63 KRW\n    *   Open: 1,420.41 KRW\n    *   High: 1,431.99 KRW\n    *   Low: 1,419.20 KRW\n*   **ExchangeRates.org.uk / Currency .ME.UK:**\n    *   Closing Rate: 1 USD = 1420 KRW (Min: 1421, Max: 1422)\n    *   Specific Rate: 1 USD = 1420.4934 KRW\n*   **Yahoo Finance:**\n    *   Close/Adjusted Close: 1 USD = 1,419.0500 KRW\n    *   Open: 1,419.0500 KRW\n    *   High: 1,431.5900 KRW\n    *   Low: 1,414.7200 KRW\n*   **Pound Sterling LIVE:**\n    *   Mid Rate: 1 USD = 1,426.3450 KRW (Open: 1421.33, High: 1430.81, Low: 1421.25, Close: 1431.44) *Note: Close rate seems different from other sources for this day.*\n*   **YCharts (Source: European Central Bank):**\n    *   Value: 0.0007 USD to 1 KRW. This is the inverse rate (KRW to USD). To get USD to KRW, you calculate

## End of notebook